# Solar Orbiter 8 SPICE tutorial, 2022-09-16

[Éric Buchlin](mailto:eric.buchlin@universite-paris-saclay.fr).

A preliminary [data analysis user's manual](https://spice-wiki.ias.u-psud.fr/doku.php/data:data_analysis_manual) is available on the SPICE wiki.
This tutorial is based on Python, but IDL users can find IDL-specific information in this manual.


## Pre-requisites

* Up-to-date web browser
* Python with a recent version of the following libraries installed:
    * [sunpy](https://sunpy.org/).
    * [astropy](https://www.astropy.org/) (should be installed automatically as a sunpy dependency)
    * [sunpy-soar](https://github.com/dstansby/sunpy-soar)
    * [sunraster](https://github.com/sunpy/sunraster)
* A FITS file viewer: [SAOImageDS9](https://sites.google.com/cfa.harvard.edu/saoimageds9), [fv](https://heasarc.gsfc.nasa.gov/ftools/fv/)...

These imports should work with no error:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units as u
import sunpy_soar
from astropy.io import fits
from sunpy.map import Map
from sunpy.net import Fido, attrs as a
from sunraster.instr.spice import read_spice_l2_fits

%matplotlib notebook
plt.rcParams["figure.figsize"] = (9, 8)  # larger default figure size

## SPICE data products overview

The data levels, FITS files, and headers are described in

* The [Data Products Description Document](https://spice.ias.u-psud.fr/spice-data/documents/SPICE-UIO-DPDD-0002-1.4-Data_Product_Description_Document.pdf) (DPDD)
* The LLDPDD for the Low-Latency (LL) files

The data levels:

* **L0**: Raw data
* **L1**: Engineering data (uncalibrated)
* **L2**: Science data (calibrated). This is the main data product you should use.
* **L3** (does not exist yet) – Higher level data: maps of fitted line parameters and of physical quantitites, images, movies.

A SPICE file name example is `solo_L2_spice-n-ras_20220302T181034_V04_100663690-000.fits`, where

* `L2` is the level
* `n-ras` is represents the product type:
    * **n-ras**: a raster scan with one of the narrow slits (2", 4" or 6")
    * **w-ras**: a raster scan with the wide slit (30")
    * **n-sit**: a sit-and-stare study with one of the narrow slits
    * **w-sit**: a sit-and-stare study with the wide slit
    * **n-exp**: a single exposure with a narrow slit that yields the entire spectrum
    * **w-exp**: a single exposure with the wide slit that yields the entire spectrum
* `20220302T181034` represents `DATE-BEG` (observation start) in UTC
* `V04` is the file version
* `100663690` is the observation ID (`SPIOBSID`)
* `000` is the index of the file (`RASTERNO`) for repeated observations within the same `SPIOBSID`:
    * For "ras" data, a FITS file contains a single raster scan (repeated exposures). If the raster is repeated, then each repeat goes in a new file.
    * For "exp" data, each exposure will end up in a different FITS file.

## Find data


### Using SOAR (web interface)

The primary source for SPICE data, as for all Solar Orbiter data, is the [Solar Orbiter Archive](http://soar.esac.esa.int/) (SOAR) at ESA.

SOAR provides a query form.

![SOAR screenshot](./fig/screenshot-soar-01.png)

SOAR has a TAP (Table Access Protocol) server (TAP is an IVOA protocol): click on the "programmatic access" icon in the left icon menu for details.
This TAP interface is used by the `sunpy_soar` Python module (developed by David Stansby), which provides SOAR access to SunPy [Fido](https://docs.sunpy.org/en/stable/guide/acquiring_data/fido.html) (Federated Internet Data Obtainer).

In [ ]:
# Attributes allow us to specify the search parameters
results_spice = Fido.search(
    a.Time('2022-03-02T00:00', '2022-03-03T00:00'),
    a.soar.Product('SPICE-N-RAS'), # same as displayed in SODA: https://www.davidstansby.com/soda/
    a.Level(2)
    )
# Display tables of results
results_spice

### SPICE data releases

All released data are included in the [data releases](https://spice.osups.universite-paris-saclay.fr/spice-data/). The latest release ([currently 2.0](https://spice.osups.universite-paris-saclay.fr/spice-data/release-2.0/release-notes.html)) contains the latest version of each file, and is regularly updated with newly-available files.

The release used in a publication should be referenced as mentioned in the release notes (this also applies to data obtained from the SOAR), together with the whole SPICE dataset ([doi:10.5270/esa-lbmdy7c](https://doi.org/10.5270/esa-lbmdy7c)).

Each release contains a CSV table of all files in the release, which can be used to find specific SPICE observations:

In [ ]:
def date_parser(string):
    try:
        return pd.Timestamp(string)
    except ValueError:
         return pd.NaT

date_columns = ['DATE-BEG','DATE', 'TIMAQUTC']
cat = pd.read_csv(
    'https://spice.osups.universite-paris-saclay.fr/spice-data/release-2.0/catalog.csv',
    date_parser=date_parser,
    parse_dates=date_columns
)

In [ ]:
# list of all columns
', '.join(cat.columns)

In [ ]:
display_cols = ['NAXIS1', 'NAXIS2', 'NAXIS3', 'NAXIS4', 'LEVEL', 'FILENAME', 'DATE-BEG']
cat[display_cols]

In [ ]:
# search according to some criteria
spice_cat = cat[
    (cat.LEVEL == 'L2') &
    (cat['DATE-BEG'] > pd.Timestamp('2022-03-02T00:00')) &
    (cat['DATE-BEG'] < pd.Timestamp('2022-03-03T00:00')) &
    (cat.STUDYTYP == 'Raster')  # rasters
]
spice_cat[display_cols]

## Download files from SOAR

Taking the result of `Fido.search()` above, we can download them (here we select only the first file, from the first (and only) provider):

In [ ]:
spice_files = Fido.fetch(results_spice[0][0], path="data/{file}")
# In case you have already downloaded the file before the tutorial
# spice_files = ['data/solo_L2_spice-n-ras_20220302T181034_V04_100663690-000.fits']
print(spice_files)

## Open FITS files

As SO remote-sensing instruments data files are regular FITS files, they can normally be opended using any FITS library, in any language. For example, here we open the downloaded SPICE file with [`astropy.io.fits`](https://docs.astropy.org/en/stable/io/fits/index.html):

In [ ]:
hdulist = fits.open(spice_files[0])
hdulist.info()

In [ ]:
# Print first HDU (Header-Data Unit) header, including global and window-specific metadata:
hdulist[0].header

In [ ]:
# And the corresponding data type and shape:
data = hdulist[0].data
type(data), data.shape


These axes dimensions correspond the the `NAXIS*` metadata in reverse order:
```
NAXIS1  =                  192 / Number of slit positions (x)
NAXIS2  =                  834 / Number of pixels along slit (y)
NAXIS3  =                   80 / Number of pixels in dispersion dimension
NAXIS4  =                    1 / Number of exposures per slit position (time)
```

So L2 files can be analysed using this FITS object, but as maps with the `Map` object of `sunpy.map`, it can be more convenient to use the `SpectrogramCube` object from [`sunraster`](https://docs.sunpy.org/projects/sunraster/en/latest/) (developed as a generalization of IRIS software, for any spectrogram data).

`sunraster` includes a SPICE file reader:

In [ ]:
from sunraster.instr.spice import read_spice_l2_fits
raster = read_spice_l2_fits(spice_files[0])

In [ ]:
raster

Keys correspond to the names of the wavelength windows on the detector. 

In case of a full-detector (non-windowed) study, they are the names of the detectors, with SW for short-wavelength and LW for long-wavelength.

One can select such a window:

In [ ]:
window = raster['Ne VIII 770 / Mg VIII 772 - SH']
window

In this case, "SH" means "short-wavelength half of the line". This is because the maximum width of SPICE spectral windows is 32 pixels on the detector, and this is most of the times not enough (all the more that we try here to catch both Ne VIII 77.0nm and Mg VIII 77.2nm). Then several adjacent windows can be used, in this case they are labelled "SH" and "LH".

However, both windows are merged in the L2 FITS files, so the content of 'Ne VIII 770 / Mg VIII 772 - SH' and 'Ne VIII 770 / Mg VIII 772 - LH' is actually the same.

In [ ]:
# As we have already seen, the 4 dimensions are (t, λ, y, x). This is the Python order, reversed from the FITS or IDL order.
window.instrument_axes

In [ ]:
# Coordinates are known in WCS (World Coordinate System)
window.wcs

In [ ]:
# For better image value normalization
from astropy.visualization import SqrtStretch, AsymmetricPercentileInterval, ImageNormalize
norm = ImageNormalize(window.data,
                      interval=AsymmetricPercentileInterval(1, 99),
                      stretch=SqrtStretch()
                     )

In [ ]:
# Show (x, y) cut in cube, at some λ that can be chosen by a slider
# Please change λ, otherwise the map will stay blank (NaN / no data)
# Note: an IDL quicklook tool exists, a Python tools are in development
window.plot(norm=norm, aspect='auto')

The bright line at the top corresponds to the bright "dumbbell", a wider part of the slit, meant to help co-alignement with imaging data.

The other horizontal lines are instrumental effects that should have been corrected.

In [ ]:
# select some central wavelength, giving a 2D (x, y) map
window_peak = window[0, 12, :, :]

In [ ]:
window_peak

In [ ]:
plt.figure()
window_peak.plot(norm=norm, aspect='auto')

In [ ]:
# An alternative is to make a sunpy Map out of the data and metadata
m_spice = Map((window_peak.data, window_peak.meta))
m_spice.plot_settings['cmap'] = plt.get_cmap('viridis')
m_spice.plot_settings['norm'] = norm

In [ ]:
plt.figure()
m_spice.plot(norm=norm, aspect=1/4)  # 1/4 because raster step is 4", about 4 times the vertical pixel size 
plt.colorbar()
plt.show()

In [ ]:
# Display the spectrum at some pixel
plt.figure()
window[0, :, 674, 60].plot()

We won't dive into line fitting, but here are some libraries that can be used:

* [`scipy.optimize`](https://docs.scipy.org/doc/scipy/reference/optimize.html)
* [`astropy.modeling`](https://docs.astropy.org/en/stable/modeling/index.html)
* Line fitting software adapted specifically to SPICE data is in development at GSFC. 

Then fitted models parameters can give line radiance, Doppler shift, line width... and allow separation of several lines, or blended lines.

**There is currently still an issue with SPICE Doppler maps, please contact us before interpreting what you see in them**.

In [ ]:
# Select and display a raster position, giving a (y, λ) detector view (but with distortion already corrected)
window_detector = window[0, :, :, 40]
plt.figure()
window_detector.plot(norm=norm, aspect='auto')

## Bonus: overplotting SPICE map on an EUI submap

In [ ]:
# SPICE average raster time time
from astropy.time import Time, TimeDelta
t = raster['Ne VIII 770 / Mg VIII 772 - SH'].time
t_av = Time(t.jd.mean(), format='jd', scale='utc')
t_av.to_value('iso')

In [ ]:
# Find corresponding FSI 17.4nm image
delta_t = TimeDelta(20 * 60, format='sec')
results_fsi = Fido.search(
    a.Time(t_av - delta_t, t_av + delta_t),
    a.soar.Product('EUI-FSI174-IMAGE'), # same as displayed in SODA
    a.Level(2)
    )
# Display tables of results
results_fsi

In [ ]:
# Download first file (if not already done)
fsi_files = Fido.fetch(results_fsi[0][0], path="data/{file}")
print(fsi_files)
# In case you have already downloaded the file before the tutorial, you can replace the download by:
# fsi_files = ['data/solo_L2_eui-fsi174-image_20220302T173017304_V01.fits']

Plot the map:

In [ ]:
# Plot a composite EUI/FSI + SPICE map
from astropy.coordinates import SkyCoord

m_fsi = Map(fsi_files[0])
bottom_left = SkyCoord(-1000 * u.arcsec, -1500 * u.arcsec, frame=m_fsi.coordinate_frame)
top_right = SkyCoord(500 * u.arcsec, 0 * u.arcsec, frame=m_fsi.coordinate_frame)
sm_fsi = m_fsi.submap(bottom_left=bottom_left, top_right=top_right)

In [ ]:
# Using a CompositeMap
comp_map = Map(sm_fsi, m_spice, composite=True)
plt.figure()
comp_map.plot()
plt.show()

# Simply plotting both maps with proper alignment
#sm_fsi.plot()
#m_spice.plot(autoalign=True)